In [1]:
%cd ..

/home/work/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='omniglot_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-3,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 50,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
# optimizer = config.init_obj('optimizer', pyro.optim)

In [12]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader)

In [13]:
trainer.train()

Train Epoch: 1 [128/17352 (1%)] Loss: -9358.930664
Train Epoch: 1 [1536/17352 (9%)] Loss: -5579.917969
Train Epoch: 1 [2944/17352 (17%)] Loss: -19323.892578
Train Epoch: 1 [4352/17352 (25%)] Loss: -5641.958008
Train Epoch: 1 [5760/17352 (33%)] Loss: 9557.472656
Train Epoch: 1 [7168/17352 (41%)] Loss: -29750.332031
Train Epoch: 1 [8576/17352 (49%)] Loss: -21824.986328
Train Epoch: 1 [9984/17352 (58%)] Loss: -26377.087891
Train Epoch: 1 [11392/17352 (66%)] Loss: -25295.343750
Train Epoch: 1 [12800/17352 (74%)] Loss: -27293.730469
Train Epoch: 1 [14208/17352 (82%)] Loss: -22941.111328
Train Epoch: 1 [15557/17352 (90%)] Loss: -22319.845703
Train Epoch: 1 [16187/17352 (93%)] Loss: -18930.851562
Train Epoch: 1 [16999/17352 (98%)] Loss: -11383.324219
    epoch          : 1
    loss           : -15607.127456562632
    val_loss       : -11188.9740234375
Train Epoch: 2 [128/17352 (1%)] Loss: -6799.135254
Train Epoch: 2 [1536/17352 (9%)] Loss: -29300.648438
Train Epoch: 2 [2944/17352 (17%)] Loss: